# CFA Data Preparation

After downloading the data from Illuminate, make sure that you filter the students that are in the Alg1, Alg2 and GEO classes and change their grade level to each of these subject course titles (example 9th to ALG1) before you run the data through this program so they have the proper grade level.

LIST STU ID LN FN GR SEC MST.RM MST TCH.TE CRS CRS.CO MST.PD CRS.DC MST.SM  

Run the query and remove all classes other than the math classes and ela classes that are of interest.

Then load cleaned file for teachers.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ela_data = pd.read_excel(r"C:\Users\derek.castleman\Desktop\eladatacfa2.xls") #New ela data
math_data = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Math CFA 2.xls") # New math data
old_data = pd.read_csv(r"C:\Users\derek.castleman\Desktop\CFA All Data\CFA Standards One.csv") #Previous combined standards file
teachers = pd.read_excel(r"C:\Users\derek.castleman\Desktop\TeachertoStudentCFA.xlsx") # Clean teacher file from query
output = "C:\\Users\\derek.castleman\\Desktop\\CFA All Data\\CFA_Three.xlsx"

In [ ]:
ela_data

In [ ]:
math_data

In [ ]:
old_data

In [ ]:
teachers

## Getting Rid of Unnecessary Columns Math

Some of the columns are unnecessary in the file from Illuminate so this will get rid of most of them.

It will also separate out the overall scores columns for later use.

In [ ]:
math_data = math_data.loc[:,~math_data.columns.str.contains('Subject') ]
math_data = math_data.loc[:,~math_data.columns.str.contains('Scope') ]
math_data = math_data.loc[:,~math_data.columns.str.contains('Title') ]
math_data = math_data.loc[:,~math_data.columns.str.contains('Description') ]
math_data = math_data.loc[:,~math_data.columns.str.contains('Date') ]
math_data = math_data.loc[:,~math_data.columns.str.contains('Time') ]
math_data = math_data.loc[:,~math_data.columns.str.contains('Mastered') ]
math_data = math_data.loc[:,~math_data.columns.str.contains('Questions')]
math_data = math_data.loc[:,~math_data.columns.str.contains('Minutes')]
math_tests = math_data.loc[:,~math_data.columns.str.contains('Author') ]

In [ ]:
first_columns_math = math_tests.iloc[:, :14] #Selecting overall test columns

In [ ]:
first_columns_math

## Fixing Math Data

The front columns will be taken out to leave only the standards column scores.

The extra information will be subtracted from the column names to leave only the standards themselves as the column names.

Then all columns will be melted leaving two columns with column names and associated values for it.

In [ ]:
# Dropping overall test and demographic information
math_standards = math_data.drop(math_data.columns[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]],axis = 1)
math_standards

In [ ]:
# Cleaning up column names
math_standards = math_standards.rename(columns=lambda x: x.split('.Content.')[-1])
math_standards = math_standards.rename(columns=lambda x: x.split('Performance Band Level')[0])
math_standards

In [ ]:
# Melting all columns down to just column names and values
math_final = math_standards.melt(['Student ID'], var_name='category', value_name='values')
math_final

In [ ]:
math_final = math_final[math_final['values'].notna()] #Dropping rows where students have no values
math_final

In [ ]:
# The final subject of Math will be added to allow for concat later on with ela
math_final['Subject'] = 'Math'
math_final

## Editing ELA Data

The same exact steps that were taken with the Math data will now be taken with the ELA data.

In [ ]:
ela_data = ela_data.loc[:,~ela_data.columns.str.contains('Subject') ]
ela_data = ela_data.loc[:,~ela_data.columns.str.contains('Scope') ]
ela_data = ela_data.loc[:,~ela_data.columns.str.contains('Title') ]
ela_data = ela_data.loc[:,~ela_data.columns.str.contains('Description') ]
ela_data = ela_data.loc[:,~ela_data.columns.str.contains('Date') ]
ela_data = ela_data.loc[:,~ela_data.columns.str.contains('Time') ]
ela_data = ela_data.loc[:,~ela_data.columns.str.contains('Mastered') ]
ela_data = ela_data.loc[:,~ela_data.columns.str.contains('Questions')]
ela_data = ela_data.loc[:,~ela_data.columns.str.contains('Minutes')]
ela_tests = ela_data.loc[:,~ela_data.columns.str.contains('Author') ]
ela_tests

In [ ]:
first_columns_ela = ela_tests.iloc[:, :14]
first_columns_ela

In [ ]:
ela_standards = ela_tests.drop(ela_data.columns[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]],axis = 1)
ela_standards

In [ ]:
ela_standards = ela_standards.rename(columns=lambda x: x.split('Literacy.')[-1])
ela_standards = ela_standards.rename(columns=lambda x: x.split('Performance Band Level')[0])
ela_standards

In [ ]:
ela_final = ela_standards.melt(['Student ID'], var_name='category', value_name='values')
ela_final

In [ ]:
ela_final = ela_final[ela_final['values'].notna()]
ela_final

In [ ]:
ela_final['Subject'] = 'ELA'
ela_final

## Fixing Teachers and Finishing Standards

Attaching the teachers to the students file. First they will be separated by subjects (advisory, ela, and math) then each will be attached to the corresponding subject (advisory to both).

The teachers will be attached to the fixed files on standards for each of the ela and math, then they will be concatenated to produce one final file that has the ela and math standards with the teachers ready for Tableau.

In [ ]:
# Selecting homeroom teachers
homeroom = teachers[teachers['Dept']== 10]
homeroom = homeroom.rename({'Teacher name': 'Advisory Teacher'}, axis=1)
homeroom = homeroom[["Student ID", "Advisory Teacher"]]
homeroom

In [ ]:
# Selecting ELA teachers
ela = teachers[teachers['Dept']== 1]
ela = ela.rename({'Teacher name': 'Subject Teacher'}, axis=1)
ela = ela[["Student ID", "Subject Teacher"]]
ela

In [ ]:
# Selecting Math teachers
math = teachers[teachers['Dept']== 2]
math = math.rename({'Teacher name': 'Subject Teacher'}, axis=1)
math = math[["Student ID", "Subject Teacher"]]
math

In [ ]:
# Matching ELA and Advisory teachers
ela_merge = pd.merge(ela, homeroom, how='left', on='Student ID')
ela_merge

In [ ]:
ela_merge = ela_merge.drop_duplicates(['Student ID','Advisory Teacher'])
ela_merge

In [ ]:
# Matching Math and Advisory teachers
math_merge = pd.merge(math, homeroom, how='left', on='Student ID')
math_merge

In [ ]:
math_merge = math_merge.drop_duplicates(['Student ID','Advisory Teacher'])
math_merge

In [ ]:
# Matching Math with Overall Scores
math_ready = pd.merge(first_columns_math, math_merge, how = 'left', on='Student ID')
math_ready

In [ ]:
# Matching ELA with Overall Scores
ela_ready = pd.merge(first_columns_ela, ela_merge, how = 'left', on='Student ID')
ela_ready

In [ ]:
# Matching Math with Fixed Standards
math_final = pd.merge(math_ready, math_final, how='left', on='Student ID')
math_final

In [ ]:
math_final = math_final.drop_duplicates(['Student ID','category', 'values'])
math_final

In [ ]:
# Matching ELA with Fixed Standards
ela_final = pd.merge(ela_ready, ela_final, how='left', on='Student ID')
ela_final

In [ ]:
ela_final = ela_final.drop_duplicates(['Student ID','category', 'values'])
ela_final

In [ ]:
# Concatenating the Math and ELA dataframes together.
final_cfa = pd.concat([math_final, ela_final])
final_cfa

In [ ]:
# CSV file is generated in case you want to look at it and make sure everything looks good.

import base64
from IPython.display import HTML

def create_download_link( df, title = "CFA Standards", filename = "CFA Standards"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final_cfa)

## Matching Standards

For the next section, the new standards for the current CFA will be merged with the standards from previous CFAs to allow for comparison.

Full outer merge is done to keep standards in common and not in common.

ELA and Math will be separated then a new column of subject will be created to deal with nulls that pop up in standards that do not overlap between different CFAs

In [ ]:
# Outer merge to include all standards
cfa_merged = pd.merge(final_cfa, old_data, how = 'outer', on = ['Student ID', 'category', 'Current Grade Level', 
                                                            'Subject Teacher', 'Advisory Teacher'])
cfa_merged

In [ ]:
# Selecting subjects that are ELA
cfa_ela = cfa_merged[(cfa_merged['Subject_x'] == 'ELA') | (cfa_merged['Subject_y'] == 'ELA')]
cfa_ela

In [ ]:
# Creating one final category of Subject to deal with nulls between missing standards
cfa_ela['Subject'] = 'ELA'
cfa_ela

In [ ]:
cfa_math = cfa_merged[(cfa_merged['Subject_x'] == 'Math') | (cfa_merged['Subject_y'] == 'Math')]
cfa_math

In [ ]:
cfa_math['Subject'] = 'Math'
cfa_math

In [ ]:
cfa_merged = pd.concat([cfa_ela, cfa_math])
cfa_merged

## Overall CFA

The original first columns that were separated at the beginning of the program will be concatenated for each subject to produce one data frame that has the overall scores for students.

Teachers for the students will be added to this dataframe in a similar manner as the standards file above.

A column of subject is also created so that we know which test we are looking at and for filtering in Tableau.

In [ ]:
ela_ready

In [ ]:
ela_ready['Subject'] = 'ELA' #Adding subject to track what test it is
ela_ready

In [ ]:
math_ready['Subject'] = 'Math' #Adding subject ot track what test it is
math_ready

In [ ]:
cfa_overall = pd.concat([ela_ready, math_ready]) # Concat both dataframes together
cfa_overall

## Final File

A final file is generated with sheets for each of the major dataframes (overall, current test standards, standards comparison).

In [ ]:
# Write all grades and assignment data to excel file
writer = pd.ExcelWriter(output)

cfa_overall.to_excel(writer, sheet_name='Overall Scores',index=False)
final_cfa.to_excel(writer, sheet_name='Current test Standards',index=False)
cfa_merged.to_excel(writer, sheet_name='Standards Comparison',index=False)

writer.save()